# REDES NEURAIS ARTIFICIAIS

[Aula 3 de Deep Learning](http://bit.ly/dn-unb03) da Engenharia de Software da UnB

In [1]:
# Installing Theano
! pip install --upgrade --no-deps git+git://github.com/Theano/Theano.git

# Installing Tensorflow
! pip install --upgrade tensorflow

# Installing Keras
! pip install --upgrade keras

# Install other packages
! pip install --upgrade pip pandas numpy matplotlib sklearn

  Cloning git://github.com/Theano/Theano.git to /tmp/pip-req-build-rhyq4m2u
  Running command git clone -q git://github.com/Theano/Theano.git /tmp/pip-req-build-rhyq4m2u
  Created wheel for Theano: filename=Theano-1.0.4+21.g8f510a1-cp37-none-any.whl size=2667466 sha256=dafe4257921d19028fe3d8a8010d1f5adb9d78a1291feea4d5386b585e8a0249
  Stored in directory: /tmp/pip-ephem-wheel-cache-ui_gaoog/wheels/ae/32/7c/62beb8371953eb20c271b3bac7d0e56e1a2020d46994346b52
Successfully built Theano
  Found existing installation: Theano 1.0.4+21.g8f510a1
    Uninstalling Theano-1.0.4+21.g8f510a1:
      Successfully uninstalled Theano-1.0.4+21.g8f510a1
Requirement already up-to-date: tensorflow in /home/administradora/anaconda3/lib/python3.7/site-packages (2.0.0)
Requirement already up-to-date: keras in /home/administradora/anaconda3/lib/python3.7/site-packages (2.3.1)


Requirement already up-to-date: pip in /home/administradora/anaconda3/lib/python3.7/site-packages (19.3.1)
Requirement already up-to-date: pandas in /home/administradora/anaconda3/lib/python3.7/site-packages (0.25.3)
Requirement already up-to-date: numpy in /home/administradora/anaconda3/lib/python3.7/site-packages (1.17.4)
Requirement already up-to-date: matplotlib in /home/administradora/anaconda3/lib/python3.7/site-packages (3.1.1)
Requirement already up-to-date: sklearn in /home/administradora/anaconda3/lib/python3.7/site-packages (0.0)


# Parte 1 - Preprocessamento dos Dados

### Importar as libs

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

### Importar o dataset

In [3]:
dataset = pd.read_csv('Churn_Modelling.csv')
X = dataset.iloc[:, 3:13].values
y = dataset.iloc[:, 13].values
print("X >>\n",X)
print("y >>\n",y)

X >>
 [[619 'France' 'Female' ... 1 1 101348.88]
 [608 'Spain' 'Female' ... 0 1 112542.58]
 [502 'France' 'Female' ... 1 0 113931.57]
 ...
 [709 'France' 'Female' ... 0 1 42085.58]
 [772 'Germany' 'Male' ... 1 0 92888.52]
 [792 'France' 'Female' ... 1 0 38190.78]]
y >>
 [1 0 1 ... 1 1 0]


### Transformando os dados categóricos


In [4]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
labelencoder_X_1 = LabelEncoder()
X[:, 1] = labelencoder_X_1.fit_transform(X[:, 1])
labelencoder_X_2 = LabelEncoder()
X[:, 2] = labelencoder_X_2.fit_transform(X[:, 2])
onehotencoder = OneHotEncoder(categorical_features = [1])
X = onehotencoder.fit_transform(X).toarray()
X = X[:, 1:]
print("X >>\n",X)

X >>
 [[0.0000000e+00 0.0000000e+00 6.1900000e+02 ... 1.0000000e+00
  1.0000000e+00 1.0134888e+05]
 [0.0000000e+00 1.0000000e+00 6.0800000e+02 ... 0.0000000e+00
  1.0000000e+00 1.1254258e+05]
 [0.0000000e+00 0.0000000e+00 5.0200000e+02 ... 1.0000000e+00
  0.0000000e+00 1.1393157e+05]
 ...
 [0.0000000e+00 0.0000000e+00 7.0900000e+02 ... 0.0000000e+00
  1.0000000e+00 4.2085580e+04]
 [1.0000000e+00 0.0000000e+00 7.7200000e+02 ... 1.0000000e+00
  0.0000000e+00 9.2888520e+04]
 [0.0000000e+00 0.0000000e+00 7.9200000e+02 ... 1.0000000e+00
  0.0000000e+00 3.8190780e+04]]


/home/administradora/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)
/home/administradora/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/_encoders.py:451: DeprecationWarning: The 'categorical_features' keyword is deprecated in version 0.20 and will be removed in 0.22. You can use the ColumnTransformer instead.
  "use the ColumnTransformer instead.", DeprecationWarning)


# Dividindo o dataset em conjunto de treinamento e conjunto de teste

In [5]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

# Feature Scaling

In [6]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

---
# Parte 2 -Vamos construir uma ANN!

## Importando o Keras


In [7]:
import keras
from keras.models import Sequential
from keras.layers import Dense

Using TensorFlow backend.


In [8]:
# Initialising the ANN
classifier = Sequential()

# Adding the input layer and the first hidden layer
classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu', input_dim = 11))

# Adding the second hidden layer
classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'sigmoid'))

# Adding the output layer
classifier.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'sigmoid'))

# Compiling the ANN
classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

# Ajustando a ANN ao conjunto de treinamento

In [9]:
classifier.fit(X_train, y_train, batch_size = 40, epochs = 100)

Epoch 1/100
8000/8000 [==============================] - 2s 205us/step - loss: 0.5943 - accuracy: 0.7960
Epoch 2/100
8000/8000 [==============================] - 0s 46us/step - loss: 0.4883 - accuracy: 0.7960
Epoch 3/100
8000/8000 [==============================] - 0s 46us/step - loss: 0.4714 - accuracy: 0.7960
Epoch 4/100
8000/8000 [==============================] - 0s 46us/step - loss: 0.4648 - accuracy: 0.7960
Epoch 5/100
8000/8000 [==============================] - 0s 46us/step - loss: 0.4591 - accuracy: 0.7960
Epoch 6/100
8000/8000 [==============================] - 0s 45us/step - loss: 0.4536 - accuracy: 0.7960
Epoch 7/100
8000/8000 [==============================] - 0s 45us/step - loss: 0.4489 - accuracy: 0.7960
Epoch 8/100
8000/8000 [==============================] - 0s 46us/step - loss: 0.4449 - accuracy: 0.7960
Epoch 9/100
8000/8000 [==============================] - 0s 47us/step - loss: 0.4414 - accuracy: 0.7960
Epoch 10/100
8000/8000 [==============================] - 0s 46

8000/8000 [==============================] - 0s 49us/step - loss: 0.3993 - accuracy: 0.8347
Epoch 80/100
8000/8000 [==============================] - 0s 50us/step - loss: 0.3993 - accuracy: 0.8351
Epoch 81/100
8000/8000 [==============================] - 0s 47us/step - loss: 0.3990 - accuracy: 0.8356
Epoch 82/100
8000/8000 [==============================] - 0s 44us/step - loss: 0.3988 - accuracy: 0.8367
Epoch 83/100
8000/8000 [==============================] - 0s 45us/step - loss: 0.3989 - accuracy: 0.8365
Epoch 84/100
8000/8000 [==============================] - 0s 44us/step - loss: 0.3987 - accuracy: 0.8367
Epoch 85/100
8000/8000 [==============================] - 0s 52us/step - loss: 0.3988 - accuracy: 0.8360
Epoch 86/100
8000/8000 [==============================] - 0s 49us/step - loss: 0.3986 - accuracy: 0.8360
Epoch 87/100
8000/8000 [==============================] - 0s 48us/step - loss: 0.3985 - accuracy: 0.8367
Epoch 88/100
8000/8000 [==============================] - 0s 48us/st

# Parte 3 - Fazendo predições e avaliando o modelo

## Prevendo os resultados com o conjunto de testes

In [10]:
y_pred = classifier.predict(X_test)
y_pred = (y_pred > 0.5)

# Criando uma Confusion Matrix

In [11]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
print(cm)

[[1540   55]
 [ 263  142]]


# Initiate activation function experimentation

# Initialising the 2nd ANN

In [12]:
del classifier

classifier = Sequential()

classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu', input_dim = 11))

classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'sigmoid'))

classifier.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'tanh'))

classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

classifier.fit(X_train, y_train, batch_size = 40, epochs = 100)

Epoch 1/100
8000/8000 [==============================] - 1s 76us/step - loss: 3.1467 - accuracy: 0.7960
Epoch 2/100
8000/8000 [==============================] - 0s 45us/step - loss: 3.1467 - accuracy: 0.7960
Epoch 3/100
8000/8000 [==============================] - 0s 44us/step - loss: 3.1467 - accuracy: 0.7960
Epoch 4/100
8000/8000 [==============================] - 0s 48us/step - loss: 3.1467 - accuracy: 0.7960
Epoch 5/100
8000/8000 [==============================] - 0s 44us/step - loss: 3.1467 - accuracy: 0.7960
Epoch 6/100
8000/8000 [==============================] - 0s 46us/step - loss: 3.1467 - accuracy: 0.7960
Epoch 7/100
8000/8000 [==============================] - 0s 45us/step - loss: 3.1467 - accuracy: 0.7960
Epoch 8/100
8000/8000 [==============================] - 0s 45us/step - loss: 3.1467 - accuracy: 0.7960
Epoch 9/100
8000/8000 [==============================] - 0s 56us/step - loss: 3.1467 - accuracy: 0.7960
Epoch 10/100
8000/8000 [==============================] - 0s 60u

8000/8000 [==============================] - 0s 46us/step - loss: 3.1467 - accuracy: 0.7960
Epoch 80/100
8000/8000 [==============================] - 0s 42us/step - loss: 3.1467 - accuracy: 0.7960
Epoch 81/100
8000/8000 [==============================] - 0s 44us/step - loss: 3.1467 - accuracy: 0.7960
Epoch 82/100
8000/8000 [==============================] - 0s 42us/step - loss: 3.1467 - accuracy: 0.7960
Epoch 83/100
8000/8000 [==============================] - 0s 42us/step - loss: 3.1467 - accuracy: 0.7960
Epoch 84/100
8000/8000 [==============================] - 0s 43us/step - loss: 3.1467 - accuracy: 0.7960
Epoch 85/100
8000/8000 [==============================] - 0s 43us/step - loss: 3.1467 - accuracy: 0.7960
Epoch 86/100
8000/8000 [==============================] - 0s 43us/step - loss: 3.1467 - accuracy: 0.7960
Epoch 87/100
8000/8000 [==============================] - 0s 44us/step - loss: 3.1467 - accuracy: 0.7960
Epoch 88/100
8000/8000 [==============================] - 0s 43us/st

In [13]:
y_pred = classifier.predict(X_test)
y_pred = (y_pred > 0.5)

# Making the Confusion Matrix

In [14]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
print(cm)

[[1595    0]
 [ 405    0]]


# Initialising the 3rd ANN

In [15]:
del classifier

classifier = Sequential()

classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'hard_sigmoid', input_dim = 11))

classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu'))

classifier.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'sigmoid'))

classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

classifier.fit(X_train, y_train, batch_size = 40, epochs = 100)

Epoch 1/100
8000/8000 [==============================] - 1s 66us/step - loss: 0.6668 - accuracy: 0.7918
Epoch 2/100
8000/8000 [==============================] - 0s 45us/step - loss: 0.6219 - accuracy: 0.7960
Epoch 3/100
8000/8000 [==============================] - 0s 44us/step - loss: 0.5879 - accuracy: 0.7960
Epoch 4/100
8000/8000 [==============================] - 0s 44us/step - loss: 0.5628 - accuracy: 0.7960
Epoch 5/100
8000/8000 [==============================] - 0s 45us/step - loss: 0.5445 - accuracy: 0.7960
Epoch 6/100
8000/8000 [==============================] - 0s 47us/step - loss: 0.5314 - accuracy: 0.7960
Epoch 7/100
8000/8000 [==============================] - 0s 44us/step - loss: 0.5224 - accuracy: 0.7960
Epoch 8/100
8000/8000 [==============================] - 0s 44us/step - loss: 0.5162 - accuracy: 0.7960
Epoch 9/100
8000/8000 [==============================] - 0s 44us/step - loss: 0.5122 - accuracy: 0.7960
Epoch 10/100
8000/8000 [==============================] - 0s 44u

8000/8000 [==============================] - 0s 46us/step - loss: 0.5059 - accuracy: 0.7960
Epoch 80/100
8000/8000 [==============================] - 0s 42us/step - loss: 0.5059 - accuracy: 0.7960
Epoch 81/100
8000/8000 [==============================] - 0s 43us/step - loss: 0.5059 - accuracy: 0.7960
Epoch 82/100
8000/8000 [==============================] - 0s 42us/step - loss: 0.5059 - accuracy: 0.7960
Epoch 83/100
8000/8000 [==============================] - 0s 49us/step - loss: 0.5059 - accuracy: 0.7960
Epoch 84/100
8000/8000 [==============================] - 0s 44us/step - loss: 0.5059 - accuracy: 0.7960
Epoch 85/100
8000/8000 [==============================] - 0s 43us/step - loss: 0.5059 - accuracy: 0.7960
Epoch 86/100
8000/8000 [==============================] - 0s 44us/step - loss: 0.5059 - accuracy: 0.7960
Epoch 87/100
8000/8000 [==============================] - 0s 43us/step - loss: 0.5059 - accuracy: 0.7960
Epoch 88/100
8000/8000 [==============================] - 0s 43us/st

In [16]:
y_pred = classifier.predict(X_test)
y_pred = (y_pred > 0.5)

# Making the Confusion Matrix

In [17]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
print(cm)

[[1595    0]
 [ 405    0]]


---
# Parte 4 - Evaluating, Improving and Tuning the ANN

## Evaluating the ANN

In [18]:
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from keras.models import Sequential
from keras.layers import Dense

def build_classifier():
    classifier = Sequential()
    classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'hard_sigmoid', input_dim = 11))
    classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'hard_sigmoid'))
    classifier.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'elu'))
    classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
    return classifier

classifier = KerasClassifier(build_fn = build_classifier, batch_size = 20, epochs = 50)
accuracies = cross_val_score(estimator = classifier, X = X_train, y = y_train, cv = 10, n_jobs = -1)
mean = accuracies.mean()
variance = accuracies.std()
print('accuracies:')
print(accuracies)   
print('\n mean:')
print(mean) 
print('\n variance:')
print(variance)

accuracies:
[0.815      0.8125     0.82749999 0.8175     0.81625003 0.83375001
 0.82749999 0.79374999 0.8175     0.79500002]

 mean:
0.815625

 variance:
0.012377522972398976


# Improving the ANN
# Dropout Regularization to reduce overfitting if needed

In [19]:
from keras.layers import Dropout

def build_classifier():
    classifier = Sequential()
    classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu', input_dim = 11))
    classifier.add(Dropout(rate = 0.1))
    classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'hard_sigmoid'))
    classifier.add(Dropout(rate = 0.1))
    classifier.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'elu'))
    classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
    return classifier

classifier = KerasClassifier(build_fn = build_classifier, batch_size = 20, epochs = 50)
accuracies = cross_val_score(estimator = classifier, X = X_train, y = y_train, cv = 10, n_jobs = -1)
mean = accuracies.mean()
variance = accuracies.std()

print('accuracies:')
print(accuracies)   
print('\n mean:')
print(mean) 
print('\n variance:')
print(variance)

accuracies:
[0.84249997 0.79000002 0.86374998 0.83875    0.86250001 0.84625
 0.84500003 0.84125    0.79874998 0.85374999]

 mean:
0.8382499992847443

 variance:
0.023460069653665268


# Tuning the ANN

In [20]:
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV
from keras.models import Sequential
from keras.layers import Dense

def build_classifier(optimizer):
    classifier = Sequential()
    classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu', input_dim = 11))
    classifier.add(Dropout(rate = 0.1))
    classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu'))
    classifier.add(Dropout(rate = 0.1))
    classifier.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'tanh'))
    classifier.compile(optimizer = optimizer, loss = 'binary_crossentropy', metrics = ['accuracy'])
    return classifier

classifier = KerasClassifier(build_fn = build_classifier)

In [ ]:
parameters = {'batch_size': [20, 34, 42],
              'epochs': [50, 100, 200],
              'optimizer': ['adam', 'rmsprop']}

grid_search = GridSearchCV(estimator = classifier,
                           scoring = 'accuracy',
                           param_grid = parameters,
                           cv = 10)

grid_search = grid_search.fit(X_train, y_train)

best_parameters = grid_search.best_params_
best_accuracy = grid_search.best_score_

Epoch 1/50
7200/7200 [==============================] - 1s 130us/step - loss: 0.6340 - accuracy: 0.7976
Epoch 2/50
7200/7200 [==============================] - 1s 93us/step - loss: 0.4406 - accuracy: 0.7989
Epoch 3/50
7200/7200 [==============================] - 1s 93us/step - loss: 0.4260 - accuracy: 0.8046
Epoch 4/50
7200/7200 [==============================] - 1s 93us/step - loss: 0.4259 - accuracy: 0.8042
Epoch 5/50
7200/7200 [==============================] - 1s 93us/step - loss: 0.4135 - accuracy: 0.8115
Epoch 6/50
7200/7200 [==============================] - 1s 105us/step - loss: 0.4077 - accuracy: 0.8118
Epoch 7/50
7200/7200 [==============================] - 1s 120us/step - loss: 0.4005 - accuracy: 0.8189
Epoch 8/50
7200/7200 [==============================] - 1s 96us/step - loss: 0.3930 - accuracy: 0.8251
Epoch 9/50
7200/7200 [==============================] - 1s 94us/step - loss: 0.3923 - accuracy: 0.8279
Epoch 10/50
7200/7200 [==============================] - 1s 95us/step 

7200/7200 [==============================] - 1s 95us/step - loss: 0.3801 - accuracy: 0.8365
Epoch 30/50
7200/7200 [==============================] - 1s 94us/step - loss: 0.3812 - accuracy: 0.8356
Epoch 31/50
7200/7200 [==============================] - 1s 93us/step - loss: 0.3760 - accuracy: 0.8375
Epoch 32/50
7200/7200 [==============================] - 1s 94us/step - loss: 0.3775 - accuracy: 0.8382
Epoch 33/50
7200/7200 [==============================] - 1s 94us/step - loss: 0.3781 - accuracy: 0.8375
Epoch 34/50
7200/7200 [==============================] - 1s 94us/step - loss: 0.3772 - accuracy: 0.8367
Epoch 35/50
7200/7200 [==============================] - 1s 95us/step - loss: 0.3828 - accuracy: 0.8329
Epoch 36/50
7200/7200 [==============================] - 1s 95us/step - loss: 0.3797 - accuracy: 0.8372
Epoch 37/50
7200/7200 [==============================] - 1s 95us/step - loss: 0.3745 - accuracy: 0.8369
Epoch 38/50
7200/7200 [==============================] - 1s 95us/step - loss

7200/7200 [==============================] - 1s 113us/step - loss: 0.3831 - accuracy: 0.8324
Epoch 8/50
7200/7200 [==============================] - 1s 91us/step - loss: 0.3786 - accuracy: 0.8319
Epoch 9/50
7200/7200 [==============================] - 1s 93us/step - loss: 0.3755 - accuracy: 0.8346
Epoch 10/50
7200/7200 [==============================] - 1s 91us/step - loss: 0.3705 - accuracy: 0.8365
Epoch 11/50
7200/7200 [==============================] - 1s 92us/step - loss: 0.3754 - accuracy: 0.8336
Epoch 12/50
7200/7200 [==============================] - 1s 93us/step - loss: 0.3739 - accuracy: 0.8346
Epoch 13/50
7200/7200 [==============================] - 1s 91us/step - loss: 0.3772 - accuracy: 0.8340
Epoch 14/50
7200/7200 [==============================] - 1s 93us/step - loss: 0.3762 - accuracy: 0.8347
Epoch 15/50
7200/7200 [==============================] - 1s 91us/step - loss: 0.3768 - accuracy: 0.8357
Epoch 16/50
7200/7200 [==============================] - 1s 94us/step - loss:

7200/7200 [==============================] - 1s 97us/step - loss: 0.3647 - accuracy: 0.8474
Epoch 36/50
7200/7200 [==============================] - 1s 94us/step - loss: 0.3634 - accuracy: 0.8460
Epoch 37/50
7200/7200 [==============================] - 1s 94us/step - loss: 0.3666 - accuracy: 0.8454
Epoch 38/50
7200/7200 [==============================] - 1s 93us/step - loss: 0.3684 - accuracy: 0.8413
Epoch 39/50
7200/7200 [==============================] - 1s 94us/step - loss: 0.3688 - accuracy: 0.8447
Epoch 40/50
7200/7200 [==============================] - 1s 95us/step - loss: 0.3667 - accuracy: 0.8447
Epoch 41/50
7200/7200 [==============================] - 1s 94us/step - loss: 0.3647 - accuracy: 0.8500
Epoch 42/50
7200/7200 [==============================] - 1s 93us/step - loss: 0.3647 - accuracy: 0.8444
Epoch 43/50
7200/7200 [==============================] - 1s 96us/step - loss: 0.3624 - accuracy: 0.8499
Epoch 44/50
7200/7200 [==============================] - 1s 123us/step - los

7200/7200 [==============================] - 1s 99us/step - loss: 3.1321 - accuracy: 0.7969
Epoch 14/50
7200/7200 [==============================] - 1s 95us/step - loss: 3.1321 - accuracy: 0.7969
Epoch 15/50
7200/7200 [==============================] - 1s 97us/step - loss: 3.1321 - accuracy: 0.7969
Epoch 16/50
7200/7200 [==============================] - 1s 96us/step - loss: 3.1321 - accuracy: 0.7969
Epoch 17/50
7200/7200 [==============================] - 1s 95us/step - loss: 3.1321 - accuracy: 0.7969
Epoch 18/50
7200/7200 [==============================] - 1s 97us/step - loss: 3.1321 - accuracy: 0.7969
Epoch 19/50
7200/7200 [==============================] - 1s 97us/step - loss: 3.1321 - accuracy: 0.7969
Epoch 20/50
7200/7200 [==============================] - 1s 95us/step - loss: 3.1321 - accuracy: 0.7969
Epoch 21/50
7200/7200 [==============================] - 1s 96us/step - loss: 3.1321 - accuracy: 0.7969
Epoch 22/50
7200/7200 [==============================] - 1s 96us/step - loss

...

7200/7200 [==============================] - 0s 62us/step - loss: 0.3879 - accuracy: 0.8297
Epoch 14/50
7200/7200 [==============================] - 0s 59us/step - loss: 0.3876 - accuracy: 0.8278
Epoch 15/50
7200/7200 [==============================] - 0s 61us/step - loss: 0.3845 - accuracy: 0.8321
Epoch 16/50
7200/7200 [==============================] - 0s 61us/step - loss: 0.3844 - accuracy: 0.8326
Epoch 17/50
7200/7200 [==============================] - 0s 57us/step - loss: 0.3807 - accuracy: 0.8344
Epoch 18/50
7200/7200 [==============================] - 0s 61us/step - loss: 0.3797 - accuracy: 0.8364
Epoch 19/50
7200/7200 [==============================] - 0s 58us/step - loss: 0.3773 - accuracy: 0.8329
Epoch 20/50
7200/7200 [==============================] - 0s 59us/step - loss: 0.3785 - accuracy: 0.8331
Epoch 21/50
7200/7200 [==============================] - 0s 60us/step - loss: 0.3821 - accuracy: 0.8278
Epoch 22/50
7200/7200 [==============================] - 0s 62us/step - loss

Epoch 42/50
7200/7200 [==============================] - 1s 69us/step - loss: 3.1535 - accuracy: 0.7956
Epoch 43/50
7200/7200 [==============================] - 0s 64us/step - loss: 3.1535 - accuracy: 0.7956
Epoch 44/50
7200/7200 [==============================] - 0s 66us/step - loss: 3.1535 - accuracy: 0.7956
Epoch 45/50
7200/7200 [==============================] - 1s 70us/step - loss: 3.1535 - accuracy: 0.7956
Epoch 46/50
7200/7200 [==============================] - 0s 67us/step - loss: 3.1535 - accuracy: 0.7956
Epoch 47/50
7200/7200 [==============================] - 0s 68us/step - loss: 3.1535 - accuracy: 0.7956
Epoch 48/50
7200/7200 [==============================] - 0s 65us/step - loss: 3.1535 - accuracy: 0.7956
Epoch 49/50
7200/7200 [==============================] - 1s 70us/step - loss: 3.1535 - accuracy: 0.7956
Epoch 50/50
7200/7200 [==============================] - 0s 64us/step - loss: 3.1535 - accuracy: 0.7956
Epoch 1/50
7200/7200 [==============================] - 3s 476us

7200/7200 [==============================] - 0s 62us/step - loss: 0.3794 - accuracy: 0.8343
Epoch 21/50
7200/7200 [==============================] - 0s 57us/step - loss: 0.3806 - accuracy: 0.8363
Epoch 22/50
7200/7200 [==============================] - 0s 59us/step - loss: 0.3798 - accuracy: 0.8325
Epoch 23/50
7200/7200 [==============================] - 0s 58us/step - loss: 0.3764 - accuracy: 0.8347
Epoch 24/50
7200/7200 [==============================] - 0s 63us/step - loss: 0.3810 - accuracy: 0.8329
Epoch 25/50
7200/7200 [==============================] - 0s 59us/step - loss: 0.3790 - accuracy: 0.8338
Epoch 26/50
7200/7200 [==============================] - 0s 59us/step - loss: 0.3787 - accuracy: 0.8321
Epoch 27/50
7200/7200 [==============================] - 0s 60us/step - loss: 0.3784 - accuracy: 0.8369
Epoch 28/50
7200/7200 [==============================] - 0s 59us/step - loss: 0.3753 - accuracy: 0.8367
Epoch 29/50
7200/7200 [==============================] - 0s 59us/step - loss

Epoch 49/50
7200/7200 [==============================] - 0s 62us/step - loss: 0.3756 - accuracy: 0.8339
Epoch 50/50
7200/7200 [==============================] - 0s 62us/step - loss: 0.3770 - accuracy: 0.8293
Epoch 1/50
7200/7200 [==============================] - 14s 2ms/step - loss: 0.7958 - accuracy: 0.7969
Epoch 2/50
7200/7200 [==============================] - 0s 59us/step - loss: 0.4805 - accuracy: 0.7956
Epoch 3/50
7200/7200 [==============================] - 0s 58us/step - loss: 0.4483 - accuracy: 0.8029
Epoch 4/50
7200/7200 [==============================] - 0s 63us/step - loss: 0.4377 - accuracy: 0.8057
Epoch 5/50
7200/7200 [==============================] - 0s 64us/step - loss: 0.4330 - accuracy: 0.8025
Epoch 6/50
7200/7200 [==============================] - 0s 58us/step - loss: 0.4238 - accuracy: 0.8049
Epoch 7/50
7200/7200 [==============================] - 0s 59us/step - loss: 0.4155 - accuracy: 0.8089
Epoch 8/50
7200/7200 [==============================] - 0s 61us/step - 

7200/7200 [==============================] - 0s 58us/step - loss: 0.3690 - accuracy: 0.8438
Epoch 28/50
7200/7200 [==============================] - 0s 62us/step - loss: 0.3703 - accuracy: 0.8415
Epoch 29/50
7200/7200 [==============================] - 0s 60us/step - loss: 0.3705 - accuracy: 0.8449
Epoch 30/50
7200/7200 [==============================] - 0s 60us/step - loss: 0.3676 - accuracy: 0.8469
Epoch 31/50
7200/7200 [==============================] - 0s 59us/step - loss: 0.3669 - accuracy: 0.8464
Epoch 32/50
7200/7200 [==============================] - 0s 62us/step - loss: 0.3620 - accuracy: 0.8471
Epoch 33/50
7200/7200 [==============================] - 0s 60us/step - loss: 0.3632 - accuracy: 0.8497
Epoch 34/50
7200/7200 [==============================] - 0s 59us/step - loss: 0.3653 - accuracy: 0.8485
Epoch 35/50
7200/7200 [==============================] - 0s 63us/step - loss: 0.3665 - accuracy: 0.8460
Epoch 36/50
7200/7200 [==============================] - 0s 62us/step - loss

# FIM

> Professor Diego Dorgam  
> [@diegodorgam](https://twitter.com/diegodorgam)